<a href="https://colab.research.google.com/github/abdullahshzd01/Attention-is-all-you-need/blob/main/i192012_i190522_i192177_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/NLP_Project'

/content/drive/My Drive/NLP_Project


In [11]:
import pandas as pd
file_name = 'English.txt'
filee=open(file_name)
English_Content=[]
for line in filee:
  English_Content.append(line.replace('\n',''))

file_name='Urdu.txt'
filee=open(file_name)
Urdu_Content=[]
for line in filee:
  Urdu_Content.append(line.replace('\n',''))

df1 = pd.Series(English_Content)
df2 = pd.Series(Urdu_Content)
data=pd.concat([df1, df2], axis=1)
data.rename(columns = {0:'English',1:'Urdu'}, inplace = True)

In [12]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [14]:
dataset = data.sample(frac=1, random_state=0)
dataset.iloc[1000:1010]

,English,Urdu
49563,Older students should attend now and there is ...,پرانے طالبعلم اب یہاں حاضری دیں اور نئے طالبعل...
49733,"""I didn't want to investigate in case it was s...","""میں اس معاملے میں تفتیش نہیں کرنا چاہتا تھا ک..."
87086,Perhaps we could simply all try it together on...,مسٹر صدر ، شاید ہم سب مل کر ایک بار پھر کوشش ک...
97372,"""If it's not against Iran, then who is it agai...","""اگر یہ ایران کے خلاف نہیں ہے تو پھر یہ کس کے ..."
11049,"The dollar index, which tracks the greenback a...",ڈالر انڈیکس ، جو اپنے ہم عمر افراد کی ٹوکری کے...
85351,Perhaps I never will.,شاید میں کبھی نہیں کروں گا۔
27506,"Yes, Mr MacCormick, because according to my in...",ہاں ، مسٹر میک میکرک ، کیوں کہ میری معلومات کے...
66891,"You'll see them again.""","آپ انہیں دوبارہ دیکھیں گے۔ """
10561,There is simply less room for blokes who put 2...,بلیک کے لئے صرف کم گنجائش ہے جو ہر ہفتے اپنے ج...
31701,Let's get rid of Zarqawi.,چلو زرقاوی سے تو جان چھوٹی ۔


In [15]:
total_sentences = 1000
test_proportion = 0.1
train_test_threshold = int( (1-test_proportion) * total_sentences)

printmd(f'## {total_sentences} "parallel sentences" will be loaded (original sentence + its translation)')
printmd(f'## {train_test_threshold} "parallel sentences" will be used to train the model')
printmd(f'## {total_sentences-train_test_threshold} "parallel sentences" will be used to test the model')

## 1000 "parallel sentences" will be loaded (original sentence + its translation)

## 900 "parallel sentences" will be used to train the model

## 100 "parallel sentences" will be used to test the model

In [16]:
def clean(string):
    # Clean the string
    string = str(string).replace("\u202f"," ") # Replace no-break space with space
    string = string.lower()
    
    # Delete the punctuation and the numbers
    for p in punctuation + "«»" + "0123456789":
        string = string.replace(p," ")
        
    string = re.sub('\s+',' ', string)
    string = string.strip()
           
    return string


# Clean the sentences
dataset["English"] = dataset["English"].apply(lambda x: clean(x))
dataset["Urdu"] = dataset["Urdu"].apply(lambda x: clean(x))

# Select one part of the dataset
dataset = dataset.values
dataset = dataset[:total_sentences]

# split into train/test
train, test = dataset[:train_test_threshold], dataset[train_test_threshold:]

# Define the name of the source and of the target
# This will be used in the outputs of this notebook
source_str, target_str = "Urdu", "English"

# The index in the numpy array of the source and of the target
idx_src, idx_tar = 1, 0

In [17]:
pd.DataFrame(dataset[1000:1010])

,0,1
